<a href="https://colab.research.google.com/github/MarcosP7635/Energy/blob/main/JupyterNotebooks/laser_diode_icf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import subprocess
running_on_colab = not "JupyterNotebooks" in subprocess.os.getcwd()
if running_on_colab:
    running_on_colab = True
    parent_dir = "/content/"
    export_data_dir = "/content/"
    import_data_dir = "/content/"
else:
    parent_dir = subprocess.os.getcwd().split("JupyterNotebooks")[0]
    sep_char = parent_dir.split("Energy")[-1]
    import_data_dir =  parent_dir + sep_char + "ImportedData" + sep_char 
    export_data_dir =  parent_dir + sep_char + "ExportedData" + sep_char
parent_dir, import_data_dir, export_data_dir

('c:\\Users\\engin\\Documents\\GitHub\\Energy\\',
 'c:\\Users\\engin\\Documents\\GitHub\\Energy\\\\ImportedData\\',
 'c:\\Users\\engin\\Documents\\GitHub\\Energy\\\\ExportedData\\')

In [ ]:
from astropy import units as units
#https://docs.astropy.org/en/stable/units/index.html
from astropy import constants as const
##https://docs.astropy.org/en/stable/constants/index.html
!pip install SymPy
import sympy
#https://docs.sympy.org/latest/
from sympy import Sum, symbols, Indexed, lambdify, integrate
from sympy.abc import *
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Consider a high power laser firing at a pellet of frozen pure deuterium. What conditions are necessary to produce net energy?
The energy produced by the fusion of deuterium, $E_{prod}$ is



In [ ]:
E_produced = Sum(Indexed('E',r) * Indexed('N',r) * Indexed('nu', r), (r,1,R))
E_produced

Sum(E[r]*N[r]*nu[r], (r, 1, R))

Consider a specific nuclear reaction that occurs after the laser is fired at the pellet, and let us denote this reaction with a positive integer $r$. The energy released by this reaction is denoted $E_r$ and the number of such reactions that will take place is $N_r$, and the energy is converted to electricity with an efficiency $\nu_r$. 

Furthermore, let us denote the energy consumed by the reaction $E_{consumed}$. It is much simpler and is given by

In [ ]:
nu_laser, E_pulse = Indexed("nu", "laser"), symbols("E_{pulse}")
E_consumed = E_pulse / nu_laser
E_consumed

E_{pulse}/nu[laser]

Where $E_{pulse}$ is the energy contained in each laser pulse and $\nu_{laser}$ is the energy efficiency of the laser (from wall-plug to pellet).

Thus the net energy produced by this apparatus, $E_{net}$

In [ ]:
E_net = E_produced - E_consumed
E_net

-E_{pulse}/nu[laser] + Sum(E[r]*N[r]*nu[r], (r, 1, R))

If we use the heat from this reaction to boil water and spin a turbine, then $\nu_r\approx0.8$. According to [Kansas State University Physics Education Research](https://perg.phys.ksu.edu/vqm/laserweb/ch-7/F7s0p11.htm), $\nu_{laser}\approx 0.2$.

It is clear that $N_r$ depends on $E_{pulse}$ so we will approximate the number of fusion reactions as a function of the laser pulse energy.

The temperature $T$ of the deuterium is not constant. It will heat up from any fusion reactions that occur and it will lose energy to the environment. What is the temperature of the pellet as a function of time? 

We now turn to the textbook The Diffuse Universe [DOI: 10.1007/978-3-662-05866-4_7](https://doi.org/10.1007/978-3-662-05866-4_7) by Dopita and Sutherland. We will assume that the deuterium is at high temperature and high density, even by the standards of astrophysical plasma. This would cause the plasma to have high optical thickness and thus be well characterized by local thermal equilibrium (LTE), and thus is primarily governed by temperature. Thus the absorption rate is similar to emission rate. Therefore, it is reasonable to model the plasma as a blackbody.  

In [ ]:
T, k, m, E, N, R = symbols("T k m E N R", 
             positive=True)
L = 4 * pi * k * R**2 * T**4 
L

4*R**2*T**4*k*pi

which describes the instantaneous power of the light emitted to the environment where $R$ is the radius of the deuterium plasma. So $L$ decreases the total kinetic energy in the plasma whereas the energy from fusion increases it. Thus we have for the energy of the plasma, $E_{plasma}$
\begin{equation}
\begin{split}
\frac{dE_{plasma}}{dt} = -L+\sum E_r\frac{dN_r}{dt}\\
\end{split}
\end{equation}
where $\frac{dN_r}{dt}$ is the fusion rate. 



We also have the question of the radius of the deuterium plasma. By dimensional analysis, we approximate the proportionality for how much the deuterium is compressed, $\Delta R$
\begin{equation}
\Delta R \propto t_0\sqrt\frac{E}{m_0}
\end{equation}
where $t$ has units of time, $E$ has units of energy, and $m_0$ has units of mass. We take $m_0$ to be the mass of the pellet, $E$ to be the energy of the laser pulse, and $t_0$  is the time it takes for the pellet to compress the deuterium. The proportionality constant must be measured experimentally.

In [ ]:
R_0, m_0, t_0, dR = symbols(
 "R_0 m_0 t_0 ΔR", positive = True)
R = R_0 - C*t_0*sympy.sqrt(E/m_0)
R

-C*sqrt(E)*t_0/sqrt(m_0) + R_0

where $C$ is the proportionality constant that must be measured experimentally

#How is the initial temperature of the deuterium related to the energy of the laser?

Let us assume that the deuterium will follow a Maxwellian distribution for probability as a function of velocity. 

In [ ]:
from sympy import pi, sqrt, exp
m = (const.m_n + const.m_p).value
k = const.k_B.value
P = (m / (k * T))**(3/2) * exp(-1 * m * v**2 / (2 * k * T)) * v**2
P *= sqrt(2 / pi) 
P

3.77542260833102e-6*sqrt(2)*T**(-1.5)*v**2*exp(-0.000121231008812884*v**2/T)/sqrt(pi)

where $m$ is the mass of a single deuterium nucleus, $T$ is the temperature of the plasma nuclei, $k$ is the Boltzmann constant, and $v$ is the velocity of a given deuterium nucleus. 

We can integrate all of the kinetic energies of the distribution to find the total energy of the plasma at a given temperature

In [ ]:
E_plasma = .5*m*integrate(P * v**2,
             (v, 0, sympy.oo))*N
E_plasma

1.46439940550764e-23*sqrt(2)*N*T

Where there are $N$ deuterium nuclei

We now have an expression for the initial temperature of the deuterium

In [ ]:
T_0 = sympy.solve(E_consumed-E_plasma, T)[0]
T_0

4.82864701069328e+22*E_{pulse}/(N*nu[laser])

Thus in general we have for the temperature of the deuterium plasma as a function of total kinetic energy, $E$

In [ ]:
T = T_0 / E_consumed * E
T

4.82864701069328e+22*E/N

In [ ]:
L =  4 * pi * k * R**2 * T**4
L

3.00223792709108e+68*pi*E**4*(-C*sqrt(E)*t_0/sqrt(m_0) + R_0)**2/N**4

Now we use a model for the fusion cross section. This will be somewhat involved, so first we download the data from experiments from the  $\text H^2(\text H^2, \text n)\text{He}^3$ and then will programmatically do so for all of the possible high energy reactions.

In [ ]:
if running_on_colab:
  from google_drive_downloader import GoogleDriveDownloader as gdd
  def download(url):
    file_id = url[url.index("/d/") + 3: -1*len("/view?usp=sharing")]
    gdd.download_file_from_google_drive(
    file_id, "/content/x4_output.txt", False)
  download("https://drive.google.com/file/d/1EqAhmeIMu1am2pZnUGC3ZSUpl8FbYOLH/view?usp=sharing")
  file_path = "/content/x4_output.txt"
else:
  file_path = import_data_dir + "x4_ddn.txt"

where $N$ is the total number of deuterium atoms at a time $t$, the total number of possible pairs of deuterium atoms that can fuse is
\begin{equation}
\begin{split}
N_{fusible\ pairs} = {N\choose2} = \frac{N!}{(N-2)!2} = \frac{N(N-1)}{2}\\
\text{For large $N$}\quad\quad N_{fusible\ pairs}\approx\frac{N^2}{2} 
\end{split}
\end{equation}

In [ ]:
N_fus_pairs  = N**2 / 2
P = ((m / (k * T))**(3/2) * 
     exp(-1 * m * v**2 / (2 * k * T)) 
      * v**2)
P *= sqrt(2 / pi)
avg_fusion_prob = sympy.integrate(
      sigma * v * P, (v, 0, sympy.oo))
fus_rate = N_fus_pairs * avg_fusion_prob
fus_rate

14112074865049.6*sqrt(2)*E**0.5*N**1.5*sigma/sqrt(pi)

which is equal to \frac{dN_r}{dt}

In [ ]:
fusion_energy = 14 * units.MeV.to(
                units.J)
power_prod = fusion_energy * fusion_rate
power_prod

31.6540512484576*sqrt(2)*E**0.5*N**1.5*sigma/sqrt(pi)

In [ ]:
power_emit = L 
power_plasma = power_prod - power_emit
power_plasma

31.6540512484576*sqrt(2)*E**0.5*N**1.5*sigma/sqrt(pi) - 3.00223792709108e+68*pi*E**4*(-C*sqrt(E)*t_0/sqrt(m_0) + R_0)**2/N**4

Where $\sigma$ is the fusion cross section.

Using this result, we have for the total number of fusion reactions,



In [ ]:
N_r = sympy.integrate(fusion_rate, (t, 0, sympy.oo))
N_r

oo*sign(sigma)

In [ ]:
fusion_rate_per_volume =

SyntaxError: ignored